In [1]:
import itertools

import numpy as np
import pandas as pd

In [2]:
thousand_separator = (lambda n:
                      '{:,.2f} р.'.format(n).replace(',', ' ')
                      if not np.isnan(n) else '- р.')

In [4]:
cur_dir = './1/2019 _ 01/'
src_df_main_sheet = pd.read_excel(
    cur_dir + 'ЯНВАРЬ_2019_ВСЕ_ПК_v1.xlsx',
    sheet_name = 0
)

In [5]:
# удаление лишних строк
src_df_main_sheet.drop(np.r_[1852:1858], axis=0, inplace=True)
src_df_main_sheet.drop([0, ], axis=0, inplace=True)

# лишние колонки
src_df_main_sheet.drop(
    src_df_main_sheet.columns[np.r_[128:145]],
    axis=1,
    inplace=True
)
src_df_main_sheet.drop(src_df_main_sheet.columns[0],
                        axis=1, inplace=True)

In [6]:
rename_icolumns = {
    0: 'Наименование ПК',
    1: 'ГРЗ ТС',
    2: 'Тип ТС'
}

rename_columns = {}
for k in rename_icolumns:
    rename_columns[src_df_main_sheet.iloc[:,k].name] = rename_icolumns[k]
#     print(
#         "Колонка ~",
#         src_df_main_sheet.columns[k],
#         "~ \n --- переименована в колонку ~",
#         rename_icolumns[k],
#     )

src_df_main_sheet.rename(columns=rename_columns, inplace=True)

In [7]:
src_df_main_sheet.set_index(['Наименование ПК', 'ГРЗ ТС', 'Тип ТС'],
                                                            inplace=True)

In [8]:
src_df_main_sheet = src_df_main_sheet.T
current_month_days = [f'2019-01-0{m}' for m in range(1, 11)]
current_month_days += [f'2019-01-{m}' for m in range(11, 32)]
levels_b = pd.MultiIndex.from_product([
    current_month_days,
    ['БПЛ км', 'БПЛ моточасы', 'РНИС км', 'РНИС моточасы']
])

src_df_main_sheet.index = levels_b
src_df_main_sheet = src_df_main_sheet.T
# src_df_main_sheet

In [9]:
# основной рабочий датафрейм
work_df = src_df_main_sheet.fillna(0)

# перестановка столбцов местами
work_df = work_df.swaplevel(1, 2, axis=0)
work_df.sort_index(inplace=True)
# work_df

In [10]:
# ОБЩАЯ СТАТИСТИКА ПО ВСЕМ ПК
grouping = work_df.groupby(level=[0, 1])

count_of_auto_total = grouping.size()
count_of_auto_total.name = 'Всего ТС'

In [15]:
def by_day_zero_distances(work_df: pd.DataFrame, count_of_auto_total: pd.Series,
                                                            report_days: list):
    """Return dataframe with data for avery day from 'repor_days' argument

    Keyword arguments:
    work_df -- ...
    count_of_auto_total -- ...
    report_days -- ...
    """
    idx = pd.IndexSlice

    columns = ['БПЛ км', 'РНИС км']
    days_to_columns = itertools.product(report_days, ['БПЛ км', 'РНИС км'])
    series_list = []

    for pair in days_to_columns:
            mask = (
                work_df.loc[
                    :,
                    idx[pair[0], pair[1]]
                ] == 0
            )
            grouping = work_df.loc[
                idx[mask, :, :],
                idx[pair[0], pair[1]]
            ]
            grouping = grouping.groupby(level=[0,1])
            series_to_append = grouping.size()
            series_to_append.name = pair[1]
            series_list.append(series_to_append)

    df = pd.concat(series_list, axis=1)

    df = df.T
    multi_idx = pd.MultiIndex.from_product([report_days, columns])
    df.index = multi_idx
    df = df.T

    df = df.reindex(count_of_auto_total.index)
    df = df.assign(e=count_of_auto_total.values)

    new_order = df.columns.unique(level=0).to_list()
    new_order.insert(0, new_order.pop())
    df = df.reindex(new_order, level=0, axis=1)

    df.rename(columns={'e': count_of_auto_total.name}, inplace=True)
    df.fillna(0, inplace=True)
    for column in df.columns:
        df[column] = df[column].astype(int)

    return df


In [17]:
days = ['2019-01-15', '2019-01-20', '2019-01-25', '2019-01-31']
zero_dist_a_day_df = by_day_zero_distances(work_df, count_of_auto_total, days)

In [18]:
zero_dist_a_day_df

Всего ТС 2019-01-15  \
                                                                 БПЛ км   
Наименование ПК Тип ТС                                                    
Волоколамский   КДМ                                       12          7   
                Мини-погрузчик с бортовым поворотом        1          1   
Егорьевский     ANТ -1000.01                               1          1   
                HAMM НD 110 VV                             1          1   
                HAMM НD 140 VV                             1          1   
                HAMME-GRW-15                               1          1   
                JCВ  3СХ                                   1          1   
                L-34                                       1          1   
                LUGONG-СLG-835                             1          1   
                S -1800-3                                  1          1   
                SCANDIA                                    1          1   
                А/КРАН                                     1          1   
                АВТОБУС                                    3          2   
                Амкадор-343В                               1          1   
                Амкадор-352С                               1          1   
                Амкадор-702Е                               1          1   
                БОРТОВОЙ                                   4          2   
                БЦМ-24.3                                   1          1   
                Беларус320                                 1          1   
                ВСН-102                                    1          1   
                ГР.СЕДЕЛЬНЫЙ ТЯГАЧ                         1          0   
                ГРУЗОВОЙ                                   1          1   
                ГС-14.02                                   1          1   
                ГУДРОНАТОР                                 1          1   
                ДЗ-122                                     1          1   
                ДЗ-122А                                    1          1   
                ДЗ-122Б-9                                  1          1   
                ДЗ-133                                     1          1   
                ДЗ-180                                     2          2   
                ДЗ-27С                                     1          1   
...                                                      ...        ...   
Серпуховский    Снегопогрузчик                             2          2   
                Трактор                                    7          7   
                Фреза                                      1          1   
                Фургон                                     1          0   
                Экскаватор                                 2          2   
                Экскаватор-планировщик                     1          1   
                Экскаватор-погрузчик                       1          1   
Талдомский      Автобус                                    1          0   
                Автогрейдер                                4          4   
                Автогудронатор                             1          1   
                Автокран                                   1          1   
                Автотопливозаправщик                       1          1   
                Асфальтоукладчик                           2          2   
                Бортовой                                   6          3   
                КДМ                                       23         11   
                Каток                                      7          7   
                Кран-манипулятор автомобильный             1          1   
                Легковое ТС                                2          1   
                Мини-погрузчик с бортовым поворотом        1          1   
                Погрузчик                                  4          4   
  

In [20]:
idx = pd.IndexSlice

zero_dist_a_day_df.loc[
    idx[:, 'КДМ'], 
    :
]

Всего ТС 2019-01-15         2019-01-20          \
                                           БПЛ км РНИС км     БПЛ км РНИС км   
Наименование ПК        Тип ТС                                                  
Волоколамский          КДМ          12          7       7          9       5   
Каширский              КДМ          15          5       2          8       6   
Коломенский            КДМ          28         11      11         14      14   
Красногорский          КДМ          24         16      14         11      18   
Луховицкий             КДМ          26          6       5         11      10   
ПК "Зарайский"         КДМ          26         15      14         18      17   
ПК "Наро-Фоминский"    КДМ          26          9      26         16      26   
ПК "Ногинский"         КДМ          21          7       7         11      11   
ПК "Павлово-Посадский" КДМ          12          6       7          7       7   
ПК "Ступинский"        КДМ          17          4       7          5      14   
ПК "Чеховский"         КДМ          18          5      18          7      18   
ПК "Шатурский"         КДМ          25         13       9         19      25   
ПК "Щёлковский"        КДМ          23          9      12         14      18   
ПК Воскресенский       КДМ          15          8       6         10       8   
ПК Высоковский         КДМ          20          8      20         12      20   
ПК Лобненский          КДМ          18          7       7          8       8   
ПК Озерский            КДМ          14          4       4          9       8   
ПК Орехово-Зуевский    КДМ          21          7       5         11      11   
ПК Раменский           КДМ          35         11      35         24      35   
ПК Рузский             КДМ          20          4       4          8       7   
ПУ "Истринский"        КДМ          20          9      15         12      14   
Серебряно Прудский     КДМ          17          5       3         14      13   
Серпуховский           КДМ          21          8       8         11      10   
Талдомский             КДМ          23         11       5         11       6   

                              2019-01-25         2019-01-31          
                                  БПЛ км РНИС км     БПЛ км РНИС км  
Наименование ПК        Тип ТС                                        
Волоколамский          КДМ             7       7          7       5  
Каширский              КДМ            12       6          7       1  
Коломенский            КДМ            16      16         12      13  
Красногорский          КДМ            19      13         14      13  
Луховицкий             КДМ            11       7          6       5  
ПК "Зарайский"         КДМ            22      23         16      12  
ПК "Наро-Фоминский"    КДМ            17      26         10      26  
ПК "Ногинский"         КДМ            14      16         10      10  
ПК "Павлово-Посадский" КДМ             7       7          9       4  
ПК "Ступинский"        КДМ            14      17          8      14  
ПК "Чеховский"         КДМ             8      18         11      18  
ПК "Шатурский"         КДМ            17      25         11       9  
ПК "Щёлковский"        КДМ            20      17         16      13  
ПК Воскресенский       КДМ            10       5         10       7  
ПК Высоковский         КДМ            18      20          4      20  
ПК Лобненский          КДМ             9       9          9       8  
ПК Озерский            КДМ            14       9          9       8  
ПК Орехово-Зуевский    КДМ            14      10         11       7  
ПК Раменский           КДМ            27      35         24      35  
ПК Рузский             КДМ            15      14          8       6  
ПУ "Истринский"        КДМ            11      14          9      20  
Серебряно Прудский     КДМ            13       7          3       1  
Серпуховский           КДМ            10       9          9       9  
Талдомский             КДМ            21       7 

In [21]:
zero_dist_a_day_df.loc[
    idx[:, 'Трактор'], 
    :
]

Всего ТС 2019-01-15         2019-01-20          \
                                            БПЛ км РНИС км     БПЛ км РНИС км   
Наименование ПК        Тип ТС                                                   
Каширский              Трактор       10         10       5         10       9   
Коломенский            Трактор        7          7       7          7       7   
Красногорский          Трактор       14         14      11         14      12   
Луховицкий             Трактор       10         10       4         10       9   
Луховицкое ДЭУ         Трактор        2          2       0          2       2   
ПК "Зарайский"         Трактор       10         10       5         10      10   
ПК "Ногинский"         Трактор        7          7       7          7       7   
ПК "Павлово-Посадский" Трактор        7          7       7          7       7   
ПК "Ступинский"        Трактор       13         13      13         13      13   
ПК "Чеховский"         Трактор       10         10      10         10      10   
ПК "Шатурский"         Трактор       11         11       2         11      11   
ПК "Щёлковский"        Трактор        7          7       3          7       5   
ПК Воскресенский       Трактор        8          8       6          8       8   
ПК Лобненский          Трактор        7          7       4          7       6   
ПК Озерский            Трактор        7          7       3          7       6   
ПК Орехово-Зуевский    Трактор       11         11       4         11       9   
ПК Раменский           Трактор        9          9       9          9       9   
ПК Рузский             Трактор        8          8       8          8       8   
ПУ "Истринский"        Трактор        8          8       8          8       8   
Серебряно Прудский     Трактор        9          9       9          9       9   
Серпуховский           Трактор        7          7       3          7       5   
Талдомский             Трактор        8          8       4          8       4   

                               2019-01-25         2019-01-31          
                                   БПЛ км РНИС км     БПЛ км РНИС км  
Наименование ПК        Тип ТС                                         
Каширский              Трактор         10       3         10       5  
Коломенский            Трактор          7       7          7       7  
Красногорский          Трактор         14      10         14      11  
Луховицкий             Трактор         10       5         10       5  
Луховицкое ДЭУ         Трактор          2       2          2       0  
ПК "Зарайский"         Трактор         10       8         10       8  
ПК "Ногинский"         Трактор          7       7          7       7  
ПК "Павлово-Посадский" Трактор          7       7          7       7  
ПК "Ступинский"        Трактор         13      13         13      13  
ПК "Чеховский"         Трактор         10      10         10      10  
ПК "Шатурский"         Трактор         11      11         11       4  
ПК "Щёлковский"        Трактор          7       4          7       3  
ПК Воскресенский       Трактор          8       5          8       5  
ПК Лобненский          Трактор          7       5          7       4  
ПК Озерский            Трактор          7       7          7       5  
ПК Орехово-Зуевский    Трактор         11       7         11       5  
ПК Раменский           Трактор          9       9          9       9  
ПК Рузский             Трактор          8       8          8       8  
ПУ "Истринский"        Трактор          8       8          8       8  
Серебряно Прудский     Трактор          9       9          9       9  
Серпуховский           Трактор          7       2          7       2  
Талдомский             Трактор          8       4          8       3